# Projeto 1 - Modelo de Classificação

### Projeto realizado por :

#### 
* Cosmin Trandafir - 57101
* Martim Baptista - 56323
* João Serafim - 56376
* Martim Paraíba - 56273
***

#### Counter de horas I guess :{}

* Cosmin Trandafir - 4h
* Martim Baptista - 
* João Serafim - 3h
* Martim Paraíba - 
***

Questoes: 
- Devemos apagar colunas que possuem demasiados valores a zero?
- Devemos testar e mostrar os testes de todos os hyperparametros ou apenas os principais?
- Diferentes resultados de melhor imputer aparecem. Qual devemos confiar?


### Neste projeto vamos usar o dataset: ***biodegradable_a.cvs*** 

In [ ]:
import pandas as pd

#Load biodegradable dataset

bio_df = pd.read_csv("biodegradable_a.csv")
bio_df.info()

## Generic functions and Imports

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, PowerTransformer
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import numpy as np

In [ ]:
def classification_scores(y_test, y_pred):
    # Evaluate the performance of the model using various metrics
    print("The Precision is: %7.4f" % precision_score(y_test, y_pred))
    print("The Recall is: %7.4f" % recall_score(y_test, y_pred))
    print("The F1 score is: %7.4f" % f1_score(y_test, y_pred))
    print("The Matthews correlation coefficient is: %7.4f" % matthews_corrcoef(y_test, y_pred))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Returns mean crossvalidation score
def evaluate_model(imputer_tuple, scaler_tuple, classifier, X_train, X_test, y_train):
    imputer = imputer_tuple[1]
    # Impute missing values
    imputer.fit(X_train)
    X_train_imputed = imputer.transform(X_train)
    X_test_imputed = imputer.transform(X_test)
    
    scaler = scaler_tuple[1]
    # Scale the test data
    scaler.fit(X_train_imputed)
    X_train_scaled = scaler.transform(X_train_imputed)
    X_test_scaled = scaler.transform(X_test_imputed)
    
    # Train the classifier
    classifier.fit(X_train_scaled, y_train)
    preds = classifier.predict(X_test_scaled)
    
    # Compute evaluation metrics for plot
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    # Compute cross-validation scores
    cv_scores = cross_val_score(classifier, X_test_scaled, y_test, cv=10)
    mean_cv_score = cv_scores.mean()
    
    # Print classificatiom metric scores
    print("Imputer: {} \nScaler: {}\nCrossValidationScore: {:.6f}\n".format(imputer_tuple[0], scaler_tuple[0], mean_cv_score))
    classification_scores(y_test, preds)
    
    return f1


    """  # Create histogram
    labels = ['Precision', 'Recall', 'F1 Score', 'MCC', 'Mean CV Score']
    scores = [precision, recall, f1, mcc, mean_cv_score]
    plt.bar(labels, scores)
    plt.title('Model Evaluation Metrics')
    plt.ylim([0.7, 1.0])
    plt.grid(axis='y')
    plt.show() """


## Inicialization 

In [ ]:
from sklearn import preprocessing
# Divide Freatures and Class columns for preprocessing
X = bio_df.iloc[:, :-1]
y = bio_df.iloc[:, -1]

# Encode string classes to a numeric value for Imputer
le = preprocessing.LabelEncoder()
df_encoded_classes = le.fit_transform(y)


## Data Imputation


***
### Combinations of 
#### Imputers: 
- SimpleImputer with mean strategy
- SimpleImputer with median strategy
- KNNImputer with 3 nearest neighbors
- KNNImputer with 5 nearest neighbors
- (Maybe put KNN =7)
- KNNImputer with 9 nearest neighbors
- KNNImputer with 11 nearest neighbors
- IterativeImputer

#### With Scalers:
- MinMaxScaler
- StandardScaler
- Normalizer
- PowerTransformer

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, df_encoded_classes, test_size=0.33, random_state=0)

# List of imputers to test (imputer_name, imputer)
imputers =[("SimpleImputer = mean", SimpleImputer(strategy='mean')), 
           ("SimpleImputer = median", SimpleImputer(strategy='median')), 
           ("KNNImputer 3",KNNImputer(n_neighbors=3)), 
           ("KNNImputer 5",KNNImputer(n_neighbors=5)), 
           ("KNNImputer 9",KNNImputer(n_neighbors=9)),
           ("KNNImputer 11",KNNImputer(n_neighbors=11)),
           # random state to use same iteration for different runs
           ("IterativeImputer",IterativeImputer(random_state=0))]

# List of scalers to test (scaler_name, scaler)
scalers = [("MinMaxScaler",MinMaxScaler()), 
           ("StandarScaler",StandardScaler()), 
           ("Normalizer",Normalizer()), 
           ("PowerTransformer",PowerTransformer())]

# List of combinations of imputer and scaler names to user in graph
combos = [imputer_name + " + " + scaler_name for imputer_name, imputer in imputers for scaler_name, scaler in scalers]

# List of mean cross validation scores for every combination of imputers and scalers
scores = []

for imputer in imputers:
    for scaler in scalers:
        # Storing scores in list
        # random state to use same iteration for different runs
        scores.append(evaluate_model(imputer, scaler, LogisticRegression(max_iter=10000, random_state=0), X_train, X_test, y_train))


In [ ]:
# figsize to adjust size 
fig, ax = plt.subplots(figsize=(20, 13))
ax.bar(combos, scores)
ax.set_title('F1 Scores')

# Set boundries for y axis 
ax.set_ylim([0.8, 1.0])
ax.grid(axis='y')

# Make labels visible
ax.set_xticklabels(combos, rotation=90, ha='center')

# Calculate and show line of max score
max_score_idx = np.argmax(scores)
highest_name, highest_score = combos[max_score_idx], scores[max_score_idx]
ax.axhline(y=scores[max_score_idx], color='r', linestyle='--', label='Max score')

ax.legend([f'Max score: {highest_name}: {highest_score:.4f}'])

plt.show()


## Feature Selection

***
### Prepare dataset using best Imputer and Scaler
 

In [ ]:
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, df_encoded_classes, test_size=0.33, random_state=42)

# Impute missing values
imputer = IterativeImputer(random_state=0)
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)



***
### Pearson Correlation between each feature and target classification

In [ ]:
# Feature selection using Pearson correlation
corr_coef = np.corrcoef(np.hstack((y_train.reshape((-1, 1)), X_train_scaled)).T)
corr_features_idx = np.where(corr_coef[0, 1:] > 0)[0]
X_train_corr = X_train_scaled[:, corr_features_idx]
X_test_corr = X_test_scaled[:, corr_features_idx]



***
### PCA

In [ ]:


# Feature selection using PCA
pca = PCA(n_components=41)
pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

tve=0 #total variance explained
for i, ve in enumerate(pca.explained_variance_ratio_):
    tve+=ve
    print("PC%d - Variance explained: %7.4f - Total Variance: %7.4f" % (i, ve, tve) )


# Define the range of number of components to test
n_components_range = range(1, 41)

# Evaluate the models using logistic regression with F1 score
lr = LogisticRegression(random_state=42)
f1_scores = []
for n_components in n_components_range:
    # Feature selection using PCA
    pca = PCA(n_components=n_components)
    pca.fit(X_train_scaled)
    X_train_pca = pca.transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    lr.fit(X_train_pca, y_train)
    y_pred = lr.predict(X_test_pca)
    f1 = f1_score(y_test, y_pred)
    f1_scores.append(f1)
    
# Find the best value of n_components based on F1 score
best_n_components = n_components_range[np.argmax(f1_scores)]

# Plot the results
plt.plot(n_components_range, f1_scores)
plt.xlabel('Number of components')
plt.ylabel('F1 score')
plt.title('PCA feature selection')
plt.axvline(best_n_components, linestyle='--', color='r', label=f'Best n_components={best_n_components}')
plt.legend()
plt.show()


# Feature selection using PCA with optimal value of components
pca = PCA(n_components=n_components)
pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Number of features to select
n_best = best_n_components

***
### SelectKBest

In [ ]:

# Feature selection using SelectKBest with F-test
k = n_best
selector = SelectKBest(f_classif, k=k)
selector.fit(X_train_scaled, y_train)
X_train_kbest = selector.transform(X_train_scaled)
X_test_kbest = selector.transform(X_test_scaled)

***
### SequentialFeatureSelection

In [ ]:
# Feature selection using Sequential Feature Selector
n_features = n_best
sfs = SequentialFeatureSelector(LogisticRegression(random_state=42, max_iter=10000),
                                n_features_to_select=n_features,
                                direction='backward',
                                scoring='f1',
                                cv=5)
sfs.fit(X_train_scaled, y_train)
X_train_sfs = sfs.transform(X_train_scaled)
X_test_sfs = sfs.transform(X_test_scaled)



***
### Putting it all together


In [ ]:
# Evaluate the models using logistic regression with F1 score
lr = LogisticRegression(random_state=42)
models = [("Pearson", X_train_corr, X_test_corr),
          ("PCA", X_train_pca, X_test_pca),
          ("SelectKBest", X_train_kbest, X_test_kbest),
          ("SequentialFeatureSelection", X_train_sfs, X_test_sfs)]

print(f'{n_best} features')
for name, X_train_fs, X_test_fs in models:
    lr.fit(X_train_fs, y_train)
    y_pred = lr.predict(X_test_fs)
    f1 = f1_score(y_test, y_pred)
    print(f"{name} with: F1 score = {f1:.4f}")

***
### Now onto selecting the best model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Best Imputer, Scaler and Selector 
imputer = IterativeImputer(random_state=42)
scaler = StandardScaler()
selector = SelectKBest(f_classif, k=22)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, df_encoded_classes, test_size=0.33, random_state=42)

# Impute missing values
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Scale features
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Feature selection using SelectKBest with F-test
selector.fit(X_train_scaled, y_train)
X_train_kbest = selector.transform(X_train_scaled)
X_test_kbest = selector.transform(X_test_scaled)


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define decision tree classifier
dtc = DecisionTreeClassifier(random_state=42)

# Define grid of hyperparameters to test
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5]
}

# Perform grid search with cross-validation
cv = GridSearchCV(dtc, param_grid=param_grid, scoring='f1', cv=5)
cv.fit(X_train_kbest, y_train)

# Print the best hyperparameters and performance score
print("Best hyperparameters:", cv.best_params_)
print("Best F1 score:", cv.best_score_)

# Evaluate the model with best hyperparameters on the test set
y_pred = cv.predict(X_test_kbest)

# Check classification scores
classification_scores(y_test, y_pred)

In [ ]:
from sklearn.metrics import classification_report

# create a Random Forest classifier object
rfc = RandomForestClassifier(random_state=42)

# define the parameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# create a GridSearchCV object
grid_search = GridSearchCV(rfc, param_grid, cv=5, scoring='f1')

# fit the GridSearchCV object to the training data
grid_search.fit(X_train_kbest, y_train)

# print the best hyperparameters found by GridSearchCV
print("Best hyperparameters:", grid_search.best_params_)

# make predictions on the test set using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_kbest)

# Check classification scores
classification_scores(y_test, y_pred)

In [ ]:
# Create a SVC classifier
svc = SVC(random_state=42)

# define the parameter grid to search over
gammas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
Cs = [0.01, 0.1, 1, 10, 100, 1000, 10000]
param_grid = {"gamma": gammas, "C": Cs}

# fit the GridSearchCV object to the training data
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring="f1")
grid_search = grid_search.fit(X_train_kbest, y_train)

# print the best hyperparameters found by GridSearchCV
print("Best hyperparameters:", grid_search.best_params_)

# make predictions on the test set using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_kbest)

# Check classification scores
classification_scores(y_test, y_pred)

In [ ]:

# define classifiers
classifiers = [('Decision Tree', DecisionTreeClassifier(criterion='gini', max_depth=8, min_samples_leaf=5, min_samples_split=2, random_state=42)),
               ('Random Forest', RandomForestClassifier(max_depth=15, min_samples_leaf=1, min_samples_split=2, n_estimators=50, random_state=42)),
               ('SVM', SVC(C=10, gamma=0.1, random_state=42)),
               ('Logistic Regression', LogisticRegression(random_state=42))]

# evaluate classifiers using precision, recall, f1, and MCC metrics
metrics = ['Precision', 'Recall', 'F1', 'MCC']
results = pd.DataFrame(columns=metrics, index=[name for name, clf in classifiers])

for name, clf in classifiers:
    clf.fit(X_train_kbest, y_train)
    y_pred = clf.predict(X_test_kbest)
    results.loc[name, 'Precision'] = precision_score(y_test, y_pred)
    results.loc[name, 'Recall'] = recall_score(y_test, y_pred)
    results.loc[name, 'F1'] = f1_score(y_test, y_pred)
    results.loc[name, 'MCC'] = matthews_corrcoef(y_test, y_pred)

# plot results
fig, ax = plt.subplots(figsize=(8, 6))
results.plot(kind='bar', ax=ax)
ax.set_ylim([0.8, 1.0])
ax.set_xlabel('Classifier')
ax.set_ylabel('Score')
ax.set_title('Classification Metrics')
plt.xticks(rotation=0)
plt.show()

# print best classifier and its F1 score
results['F1'] = results['F1'].astype(float)
best_classifier = results['F1'].idxmax()
best_score = results.loc[best_classifier, 'F1']
print(f'The best classifier is {best_classifier}, with the F1 score of {best_score:.5f}')